In [ ]:
# Importing necessary libraries
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, LSTM, Dense, Embedding
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

In [ ]:
# Load and preprocess the dataset
max_features = 10000  # Number of words to consider as features
maxlen = 100  # Cut texts after this number of words (among top max_features most common words)

In [ ]:
# (a) Preprocessing of the Data
print("Loading data...")
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

print(f"Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

Loading data...
17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Pad sequences (samples x time)


In [ ]:
# (b) Divide data into training and testing data set
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

Training data shape: (25000, 100)
Testing data shape: (25000, 100)


In [ ]:
# (c) Build the GRU Model
def build_gru_model():
    model = Sequential()
    model.add(Embedding(max_features, 128))
    model.add(GRU(128, return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# Build the LSTM Model for comparison
def build_lstm_model():
    model = Sequential()
    model.add(Embedding(max_features, 128))
    model.add(LSTM(128, return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# (d) Training the GRU Model
gru_model = build_gru_model()
print("Training GRU model...")
gru_model.fit(X_train, y_train, batch_size=32, epochs=5, validation_split=0.2)

# Training the LSTM Model
lstm_model = build_lstm_model()
print("Training LSTM model...")
lstm_model.fit(X_train, y_train, batch_size=32, epochs=5, validation_split=0.2)

Training GRU model...
Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 98s 153ms/step - accuracy: 0.6823 - loss: 0.5638 - val_accuracy: 0.8460 - val_loss: 0.3574
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 96s 153ms/step - accuracy: 0.8964 - loss: 0.2523 - val_accuracy: 0.8506 - val_loss: 0.3431
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 143s 154ms/step - accuracy: 0.9483 - loss: 0.1485 - val_accuracy: 0.8516 - val_loss: 0.3812
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 142s 155ms/step - accuracy: 0.9731 - loss: 0.0842 - val_accuracy: 0.8380 - val_loss: 0.4827
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 140s 152ms/step - accuracy: 0.9861 - loss: 0.0479 - val_accuracy: 0.8390 - val_loss: 0.6555
Training LSTM model...
Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 118s 184ms/step - accuracy: 0.7314 - loss: 0.5286 - val_accuracy: 0.8328 - val_loss: 0.3807
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 143s 186ms/step - accuracy: 0.8956 - loss: 0.2656 - val_accuracy: 0.8414 - val_loss: 0.3747
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
# (e) Evaluate Models
gru_score, gru_acc = gru_model.evaluate(X_test, y_test)
lstm_score, lstm_acc = lstm_model.evaluate(X_test, y_test)

print(f"GRU Model Accuracy: {gru_acc * 100:.2f}%")
print(f"LSTM Model Accuracy: {lstm_acc * 100:.2f}%")

782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.8361 - loss: 0.6881
782/782 ━━━━━━━━━━━━━━━━━━━━ 46s 59ms/step - accuracy: 0.8285 - loss: 0.6015
GRU Model Accuracy: 83.66%
LSTM Model Accuracy: 83.18%


In [ ]:
# (f) Text Generation (Simplified Example)

def generate_text(model, start_text, word_index, index_word, max_len=100):
    for _ in range(max_len):
        encoded_input = np.array([word_index.get(word, 0) for word in start_text])
        encoded_input = sequence.pad_sequences([encoded_input], maxlen=maxlen)
        prediction = model.predict(encoded_input)
        next_word = index_word.get(np.argmax(prediction), '?')
        start_text += ' ' + next_word
        if next_word == 'end':
            break
    return start_text

In [ ]:
word_index = imdb.get_word_index()
index_word = {index: word for word, index in word_index.items()}

start_text = "the movie was"
generated_text_gru = generate_text(gru_model, start_text, word_index, index_word)
generated_text_lstm = generate_text(lstm_model, start_text, word_index, index_word)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━

In [ ]:
print("Text Generated by GRU Model:", generated_text_gru)
print("Text Generated by LSTM Model:", generated_text_lstm)

Text Generated by GRU Model: the movie was ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
Text Generated by LSTM Model: the movie was ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?


In [ ]:
lstm_score, lstm_acc = lstm_model.evaluate(X_test, y_test)
gru_score, gru_acc = gru_model.evaluate(X_test, y_test)

# Print the accuracy results
print(f"LSTM Model Accuracy: {lstm_acc * 100:.2f}%")
print(f"GRU Model Accuracy: {gru_acc * 100:.2f}%")

782/782 ━━━━━━━━━━━━━━━━━━━━ 47s 59ms/step - accuracy: 0.5133 - loss: 0.6930
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.8361 - loss: 0.6881
LSTM Model Accuracy: 51.15%
GRU Model Accuracy: 83.66%
